In [1]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook

%matplotlib inline
%load_ext autoreload
%autoreload 2

from comet_ml import Experiment

##### TODO
- Test the ``chess`` library
    - Convert to numpy array
    - Gif 
- Use pre-existing GameAI
- Create GameAI with minimax techniques
- Evaluation function for each move
- Move recommendation "à la" AlphaGo, MCTS ?
- Notebook interface to play
- Adaptive GameAI



# Test of the ``chess`` library
https://python-chess.readthedocs.io/en/latest/



# Playground

In [2]:
import sys
sys.path.append("../")

from beth.game import Game
from beth.players.random_player import RandomPlayer
from beth.players.human_player import HumanPlayer
from beth.players.sequence import SequenceGame,SequencePlayer

In [7]:
black = RandomPlayer()
white = RandomPlayer()

# white = HumanPlayer()
# black = HumanPlayer()

game = Game(white,black)

In [8]:
game.reset_game()
game.run()

Output()

In [5]:
game.board.is_seventyfive_moves()

True

# Game exploration

In [3]:
games = pd.read_csv("../data/raw/kaggle/games.csv")

In [4]:
games.shape

(20058, 16)

In [5]:
"start " + games["moves"].head()

0    start d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ N...
1    start d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Q...
2    start e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 a...
3    start d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2...
4    start e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be...
Name: moves, dtype: object

In [6]:
game = games.iloc[2]

seq_str = game["moves"]
winner = game["winner"]
victory_status = game["victory_status"]

In [7]:
seq = SequenceGame(seq_str,0.1,victory_status,winner)
game = Game(seq.white,seq.black)
seq_str

'e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6 bxc6 Ra6 Nc4 a4 c3 a3 Nxa3 Rxa3 Rxa3 c4 dxc4 d5 cxd5 Qxd5 exd5 Be6 Ra8+ Ke7 Bc5+ Kf6 Bxf8 Kg6 Bxg7 Kxg7 dxe6 Kh6 exf7 Nf6 Rxh8 Nh5 Bxh5 Kg5 Rxh7 Kf5 Qf3+ Ke6 Bg4+ Kd6 Rh6+ Kc5 Qe3+ Kb5 c4+ Kb4 Qc3+ Ka4 Bd1#'

In [8]:
seq.victory_status

'mate'

In [9]:
game.run()

Output()

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

# Experiment 1 using LSTMs
https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html

In [14]:
from beth.models.dataset import Dataset
from beth.utils import make_experiment
from beth.models.lstm import LSTMModel
from beth.players.ai_player import AIPlayer

## Init classes

In [15]:
games = pd.read_csv("../data/raw/kaggle/games.csv")
moves = "start " + games["moves"]
moves.head()

0    start d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ N...
1    start d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Q...
2    start e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 a...
3    start d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2...
4    start e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be...
Name: moves, dtype: object

In [16]:
moves[0]

'start d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4'

In [17]:
dataset = Dataset(moves)

In [18]:
len(dataset.uniq_words)

4448

## Train loop

In [37]:
experiment = make_experiment("../.env","Simple LSTM",["NLP","LSTM"])

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/theolvs/beth/2f28f4cbecb646179c6a3d4a0ca2b444



In [38]:
params = {
    "sequence_length":10,
    "batch_size":32,
    "max_epochs":1,
    "lr":0.01,
    "lstm_size":128,
    "embedding_dim":128,
    "num_layers":3,
}

In [39]:
model = LSTMModel(dataset,**params)

In [40]:
experiment.log_parameters(params)
model.fit(**params)
# experiment.end()


... Stopped training in notebook


In [41]:
model.save_weights("lstm_20122020.pth")

In [42]:
experiment.log_text("Learning rate does not seem to accelerate anything here, let's stick to around 0.001")

{'web': 'https://www.comet.ml/api/asset/download?assetId=288a9cff5cf546748396d233c2cd1429&experimentKey=2f28f4cbecb646179c6a3d4a0ca2b444',
 'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=288a9cff5cf546748396d233c2cd1429&experimentKey=2f28f4cbecb646179c6a3d4a0ca2b444',
 'assetId': '288a9cff5cf546748396d233c2cd1429'}

In [43]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/theolvs/beth/2f28f4cbecb646179c6a3d4a0ca2b444
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [1242] : (3.267238140106201, 8.412087440490723)
COMET INFO:   Others:
COMET INFO:     Name : Simple LSTM
COMET INFO:   Parameters:
COMET INFO:     batch_size      : 32
COMET INFO:     embedding_dim   : 128
COMET INFO:     lr              : 0.01
COMET INFO:     lstm_size       : 128
COMET INFO:     max_epochs      : 1
COMET INFO:     num_layers      : 3
COMET INFO:     sequence_length : 10
COMET INFO:   Uploads:
COMET INFO:     code                     : 1 (3 KB)
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (

In [136]:
p = model.predict("start",1,as_proba = True)

In [137]:
p

,0
O-O,8.900061e-04
start,5.253599e-04
Nf3,1.236997e-01
e4,2.231918e-01
Nf6,1.312937e-07
...,...
Rexe1+,5.741276e-13
R5a2,1.169191e-12
e5#,7.000148e-13
R1g6+,1.899962e-12


In [48]:
p[0].sort_values(ascending = False).head(10)

e5     0.081328
c5     0.065016
d5     0.050288
Nf3    0.043862
f4     0.026393
Nc3    0.025715
c4     0.021629
Nc6    0.019267
c3     0.018952
e6     0.018057
Name: 0, dtype: float64

## Test in games

In [19]:
model = LSTMModel(dataset)

In [20]:
model.load_weights("lstm_20122020.pth")

In [25]:
# black = RandomPlayer()
# white = RandomPlayer()

white = AIPlayer(brain=model,wait = 1)
black = AIPlayer(brain=model,wait = 1)

# white = HumanPlayer()
# black = HumanPlayer()

game = Game(white,black)

In [26]:
game.run()

Output()

In [118]:
p = model.predict_next(game)[0]

In [126]:
p.loc[game.get_legal_moves_san()].sort_values(ascending = False)

c5      0.021046
exd4    0.018118
d5      0.014802
Nd5     0.012906
a6      0.008370
Nc6     0.008196
b6      0.008127
d6      0.007369
a5      0.006705
b5      0.006517
h5      0.006318
g5      0.005129
c6      0.004801
Nxe4    0.003987
Bd6     0.003517
h6      0.002916
g6      0.002523
Bb4+    0.001906
Ng4     0.001771
Bc5     0.001545
Qe7     0.001522
Be7     0.001458
Nh5     0.000916
Ba3     0.000718
Na6     0.000466
Ke7     0.000290
Rg8     0.000162
Ng8     0.000003
Name: 0, dtype: float64

In [67]:
stack[0]

Move.from_uci('e2e4')

In [68]:
x = stack[0]

In [73]:
game.board.san(stack[1])

AssertionError: san() and lan() expect move to be legal or null, but got e7e5 in rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2